# PS2: Matrix Inverse (8 Points)

### Problem Statement
In this assignment you are going to wite a kernel to compute a matrix inverse which we can then use to solve a linear system. We have provided some starter code for you, and your job is to fill in the missing code as specified by the `#TODO#` blocks in the code. You can either just work in the ipynb OR you can work locally with the various files in this folder. If you don't have a GPU on your machine you will need to upload the ipynb to [Google Colaboratory](https://colab.research.google.com/) and work there.

### Functions You'll Need To Implement
All functions you need to implement are in `util.h` and that is the only file you need to submit to courseworks! Please do not change function definitions. Also be careful with your syncs!

**`matrix_inverse_inner` (5 points):** this device function will do all of the heavy lifting and actually implement the matrix inverse function. It takes in the input matrix and outputs the inverse. You should assume these pointers are already in shared memory. There is also an additional input of additional shared temporary memory which you may or may not need to use (and you will get to specify how big it is in later functions). You can implement this in any way, but/and the simplest way to do this is through the use of the Gauss Jordan elimination method. You can see some graphical examples of "elementary row operations" [at this link](https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html). But/and in short you can simply walk through the rows in order (aka do not swap any rows and just do them in order) and for each row divide it to ensure that it is leading with a 1 and then subtract down along the rest of the rows to ensure they are all leading with a 0. If you repeat this and move down and to the right through the matrix you will end up with the identity in place of the original matrix and the inverse to the right! You can assume that the input matrix does not have a zero in the top left corner and is of a relatively small size (aka less than 30x30) and that it is a square matrix. The function input `matrix_dim` is the number of rows (and since the matrix is square also the number of columns) of the input matrix.

Note: there are a number of ways to actually implement it. Do whatever you think is simplest and easiest first. Then see if you can optimize it!

**`matrix_inverse_kernel` (2 points):** this kernel will call the `_inner` function and will and should move the device matrix into shared memory, make sure the computation occurs in shared memory, and return the value to device memory. You will note that we specified dynamic shared memory. Please use that and make sure to allocate enough of it in the host function to support however much your device function needs! As with `_inner`, the function input `matrix_dim` is the number of rows (and since the matrix is square also the number of columns) of the input matrix.

**`matrix_inverse` (1 point):** the host function will call the `_kernel` and will move the input host matrix into device memory, launch the kernel (with dynamic shared memory), and return the solution to host memory. As with `_inner` and `_kernel`, the function input `matrix_dim` is the number of rows (and since the matrix is square also the number of columns) of the input matrix.

### Example Output
Please see below for inline example output from Python.

### Submission
Once you are done, download and submit (or just submit if you are working locally) your `ipynb` or `util.h` file to **Courseworks**! Sadly Gradescope does not support GPU autograders at this time.

### Notes and Hints
+ **DO NOT CHANGE FUNCTION DEFINITIONS** or you will break our grading scripts
+ See the syllabus for our course collaboration policy (long story short you are welcome to collaborate at a high level but please do not copy each others code).
+ Please reach out on Slack with any and all questions!

In [ ]:
# make sure CUDA is installed
!nvcc --version

In [ ]:
# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

In [ ]:
# CUDA in Jupyter helpers
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

## Your Code Goes Here:

In [ ]:
%%cuda_group_save -n util.h -g default

// the GPU device function
//
// Hints: actually solve the matrix inverse!
//
__device__
void matrix_inverse_inner(float *s_input, float *s_output, float *s_temp, const int matrix_dim){
  // first set up the matrix with identity next to it
  // #TODO


  // Next we are going to guassian elimination walking down the matrix (assuming no leading 0s).
  // We therefore use the columns in order as the pivot column for each pivot we need to rescale
  // that row so that the pivot value is 1 THEN for all other row values we need to add a multiple
  // of the NEW pivot row value such that we transorm the other row pivot column value to 0.
  // See https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html
  //
  // Note if you would prefer to use another method that is fine but/and this is the method
  // we have a solution for and are prepared to help you with!
  for (unsigned pivRC = 0; pivRC < matrix_dim; pivRC++){
      //
      // #TODO
      //
  }

  // Make sure to write the result to the output
  // # TODO
}

// the GPU kernel
//
// Hints: set up shared memory, run the _inner, clean up shared memory
//
__global__
void matrix_inverse_kernel(float *d_input, float *d_output, const int matrix_dim){
  // get shared pointers
  extern __shared__ float s_dynShared[];
  // #TODO

  // copy the data into shared memory
  // #TODO

  // run the code
  // #TODO

  // copy the memory back out
  // #TODO
}

// the host function
//
// Hints: set up GPU memory, run the kernel, clean up GPU memory
//
__host__
void matrix_inverse(float *h_input, float *h_output, const int matrix_dim){
  // transfer memory to the device
  // #TODO

  // run the kernel
  // #TODO

  // transfer data back to the host
  // #TODO
}

## Code to run and test your code follows:

In [ ]:
%%cuda_group_save -n run.cu -g default
#include <stdio.h>
#include "util.h"

__host__
void printMat(float *mat, const int matrix_dim){
    // loop through row by row and print
    for (int r = 0; r < matrix_dim; r++){
        for (int c = 0; c < matrix_dim; c++){
            printf("%f ",mat[r + c*matrix_dim]);
        }
        // Newline for new row
        printf("\n");
    }
    // Newline for end of print
    printf("\n");
}

__host__
int runTest(float *h_input, float *h_output, const int matrix_dim){
  // print the input matrix
  printMat(h_input,matrix_dim);

  // run the main function
  matrix_inverse(h_input,h_output,matrix_dim);

  // print the final result
  printMat(h_output,matrix_dim);
}

__host__
int main() {

  // initialize the first test matrix
  const int matrix_dim = 5;
  const int matrix_dim_sq = matrix_dim*matrix_dim;
  float *h_input = (float *)malloc(matrix_dim_sq*sizeof(float));
  float *h_output = (float *)malloc(matrix_dim_sq*sizeof(float));
  for (int c = 0; c < matrix_dim; c++){
      for (int r = 0; r < matrix_dim; r++){
          h_input[r + c*matrix_dim] = (r == c) ? 2 : 0;
      }
  }
  // run the test
  runTest(h_input,h_output,matrix_dim);

  // now do the second test
  for (int c = 0; c < matrix_dim; c++){
      for (int r = 0; r < matrix_dim; r++){
          h_input[r + c*matrix_dim] = r >= c ? 7 : 0;
      }
  }
  runTest(h_input,h_output,matrix_dim);

  return 0;
}

In [ ]:
%cuda_group_run -g default

The result of your code above should match the Python code below!

In [ ]:
import numpy as np
test1 = 2*np.eye(5)
print(test1)
print(np.linalg.inv(test1))
test2 = 7*np.tri(5)
print(test2)
print(np.linalg.inv(test2))